# Zeta Function

In [1]:
using FastArrays

function Bernoulli(m::Integer)
  B = FastArray(0:m){Float64}(:)
  fill!(B, 0)
  B[0] = 1
  for n in 1:m
    B[n] = 1 - sum(binomial(n, k) * B[k] / (n-k+1) for k in 0:n-1)
  end
  B[m]
end

function opt_truc(f::Function, n::Integer=1)
    s = 0 # sum of series
    a = f(n) # a[n]
    a_post = f(n+1) # a[n+1]
    while abs(a_post) < abs(a)
        s += a
        n += 1
        a = a_post
        a_post = f(n+1)
    end
    s
end

function ζ1(s, N)
  sum(1/n^s for n = 1:N)
end

function ζ2(s, N)
  sum(1/n^s for n = 1:N) + 1/((s-1)*N^(s-1)) - 1/(2*N^s) +
  opt_truc(n -> Bernoulli(2n) / gamma(2n+1) / N^(s+2n-1) * gamma(s+2n-1) / gamma(s))
end

ζ2 (generic function with 1 method)

输出结果左边是ζ1, 右边是ζ2

In [2]:
err = zeros(5,2)
for N = 1:5
  err[N, 1] = zeta(2) - ζ1(2, N)
  err[N, 2] = zeta(2) - ζ2(2, N)
end
err

5×2 Array{Float64,2}:
 0.644934   0.0116007  
 0.394934  -1.54986e-5 
 0.283823   2.40479e-8 
 0.221323  -3.9371e-11 
 0.181323   1.12133e-13

# Airy Function

In [3]:
a(n) = (-3/4)^n * gamma(n + 5/6) * gamma(n + 1/6) / 2π / gamma(n+1)

function Ai1(x)
  exp(-2/3 * x^(3/2)) / (2*√π * x^(1/4)) *
  opt_truc(n -> a(n) * x^(-3n/2), 0)
end

function Ai2(x)
  Ai1(x) +
  1im * exp(2/3 * x^(3/2)) / (2*√π * x^(1/4)) *
  opt_truc(n -> (-1)^n * a(n) * x^(-3/2 * n), 0)
end

Ai2 (generic function with 1 method)

第一行是airy函数精确值, 第二行是一个级数结果, 第三行是两个级数结果。比较二三行与第一行的值，可以看出符合题目的描述。

In [4]:
κ = [π/2, 2π/3, 5π/6]
x = 5exp(κ*im)
println(airy(x))
println(Ai1.(x))
println(Ai2.(x))

Complex{Float64}[29.9015+21.6778im,284.832-164.448im,-3.82315-36.7335im]
Complex{Float64}[29.9015+21.6778im,284.832-164.448im,-3.82281-36.7344im]
Complex{Float64}[29.9024+21.678im,284.832-164.448im,-3.82315-36.7335im]
